# Step02 検索実行 (Question-Answering) - 回答例

ここでは、ユーザーからの質問に対し、Step01 で作成したテキストとベクトル (Embedding) のデータ (```embeddings_data.csv```) の中から答えを探して回答する仕組みを構築していきましょう。

準備として、OpenAI Python Library を Azure OpenAI Service に接続するための設定をおこないます。

In [ ]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base =  os.environ["OPENAI_API_BASE"]
openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]

embedding_model = "text-similarity-curie-001-deploy"
qa_model = "text-davinci-003-deploy"

Knowledge Base (```embeddings_data.csv```) から、全ドキュメントの Embedding 情報を読み込みます。<br>
列として、```text``` (本文), ```n_tokens``` (トークン数), ```embeddings``` (ベクトルのリスト) が含まれていることを確認してください。

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv("embeddings_data.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)

df.head()

質問に渡すコンテキストとして、質問と近いテキストを 1800 トークンを超えない範囲で取得してください。

ヒント
- 質問を Embedding します (Embedding 方法は、Step01 を参考にしてください)
- ベクトルどうしの距離を測る関数はさまざま存在します。<br>
例えば、```openai.embeddings_utils.distances_from_embeddings``` も使用できます。(この関数の詳細は [こちら](https://github.com/openai/openai-python/blob/main/openai/embeddings_utils.py) を参照してください。)
- 上述の通り、トークン数の情報は ```n_tokens``` 列に含まれています。

In [ ]:
from openai.embeddings_utils import distances_from_embeddings

def create_context(
    question, df, max_len=1800,
):
    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine=embedding_model)["data"][0]["embedding"]

    # Get the distances from the embeddings
    df["distances"] = distances_from_embeddings(q_embeddings, df["embeddings"].values, distance_metric="cosine")

    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values("distances", ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row["n_tokens"] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

In [ ]:
# 動作確認
create_context(
    "木曽川に建設された発電所のうち、1924年(大正13年)に完成したダム式の発電所は?",
    df=df,
)

つぎに、上記で抽出した (1800 token を超えない) コンテキストを使って、下記のプロンプトを渡して completion を実行します。<br>
返ってきた答えが、最終的な回答です。

----------

```
以下のテキストを使って下記の質問に答えてください。もし答えがない場合には、「私は知らない」と答えてください。

コンテキスト: {context}

---

質問: {question}
答え:
```

In [ ]:
def answer_question(
    df,
    model,
    question,
    max_len=1800,
    max_tokens=150,
    stop_sequence=None
):
    context = create_context(
        question,
        df,
        max_len=max_len,
    )
    # Create a completions using the question and context
    response = openai.Completion.create(
        prompt=f"以下のテキストを使って下記の質問に答えてください。もし答えがない場合には、「私は知らない」と答えてください。\n\nコンテキスト: {context}\n\n---\n\n質問: {question}\n答え:",
        temperature=0,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=stop_sequence,
        engine=qa_model,
    )
    return response["choices"][0]["text"].strip()

下記の質問で試してみてください。<br>
これらの質問は、標準の GPT-3.5 (```text-davinci-003```) では答えられない内容です。

1. 「木曽川に建設された発電所のうち、1924年(大正13年)に完成したダム式の発電所は?」<br>
  答え : 大井発電所
2. 「1948年に、占領軍総司令部民間情報局が静岡に設立したのは?」<br>
  答え : アメリカ文化センター
3. 「今日は何曜日?」<br>
  答え : 私は知らない

In [ ]:
answer_question(df, question="木曽川に建設された発電所のうち、1924年(大正13年)に完成したダム式の発電所は?", model=qa_model)

In [ ]:
answer_question(df, question="1948年に、占領軍総司令部民間情報局が静岡に設立したのは?", model=qa_model)

In [ ]:
answer_question(df, question="今日は何曜日", model=qa_model)